# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = True
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 3
iteration = "-3"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_running_test"
LOAD_LOCATION = "./data/models/model_best_test"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION + iteration)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

initializing model with random weights


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2000
loss: 0.6976029872894287
---------------
Batch 2/2000
loss: 0.6932961940765381
---------------
Batch 3/2000
loss: 0.6851388216018677
---------------
Batch 4/2000
loss: 0.7009849548339844
---------------
Batch 5/2000
loss: 0.6885110139846802
---------------
Batch 6/2000
loss: 0.690210223197937
---------------
Batch 7/2000
loss: 0.6974461078643799
---------------
Batch 8/2000
loss: 0.6874875426292419
---------------
Batch 9/2000
loss: 0.6936413645744324
---------------
Batch 10/2000
loss: 0.7028606534004211
---------------
Batch 11/2000
loss: 0.6865960955619812
---------------
Batch 12/2000
loss: 0.6942498087882996
---------------
Batch 13/2000
loss: 0.7015674114227295
---------------
Batch 14/2000
loss: 0.6807398796081543
---------------
Batch 15/2000
loss: 0.6825631260871887
---------------
Batch 16/2000
loss: 0.6957496404647827
---------------
Batch 17/2000
loss: 0.6879881024360657
---------------
Batch 18/2000
loss: 0.6937398314476013
---------------
Batch 19/2

---------------
Batch 150/2000
loss: 0.6619741320610046
---------------
Batch 151/2000
loss: 0.5697988867759705
---------------
Batch 152/2000
loss: 0.6885290145874023
---------------
Batch 153/2000
loss: 0.6084033250808716
---------------
Batch 154/2000
loss: 0.6553246974945068
---------------
Batch 155/2000
loss: 0.6764626502990723
---------------
Batch 156/2000
loss: 0.6180123090744019
---------------
Batch 157/2000
loss: 0.5586798191070557
---------------
Batch 158/2000
loss: 0.6485776305198669
---------------
Batch 159/2000
loss: 0.7015882730484009
---------------
Batch 160/2000
loss: 0.641255259513855
---------------
Batch 161/2000
loss: 0.6533246636390686
---------------
Batch 162/2000
loss: 0.7109478116035461
---------------
Batch 163/2000
loss: 0.6415504217147827
---------------
Batch 164/2000
loss: 0.6039326190948486
---------------
Batch 165/2000
loss: 0.645125150680542
---------------
Batch 166/2000
loss: 0.6397889852523804
---------------
Batch 167/2000
loss: 0.62415093183

---------------
Batch 297/2000
loss: 0.6453614830970764
---------------
Batch 298/2000
loss: 0.6479699015617371
---------------
Batch 299/2000
loss: 0.6033727526664734
---------------
Batch 300/2000
loss: 0.6409482955932617
---------------
Batch 301/2000
loss: 0.6266986727714539
---------------
Batch 302/2000
loss: 0.5856637954711914
---------------
Batch 303/2000
loss: 0.624332845211029
---------------
Batch 304/2000
loss: 0.6022745966911316
---------------
Batch 305/2000
loss: 0.5878143906593323
---------------
Batch 306/2000
loss: 0.6306381821632385
---------------
Batch 307/2000
loss: 0.5786644220352173
---------------
Batch 308/2000
loss: 0.658431351184845
---------------
Batch 309/2000
loss: 0.6164413690567017
---------------
Batch 310/2000
loss: 0.6563915610313416
---------------
Batch 311/2000
loss: 0.5971319079399109
---------------
Batch 312/2000
loss: 0.6101046800613403
---------------
Batch 313/2000
loss: 0.6348301768302917
---------------
Batch 314/2000
loss: 0.58562564849

---------------
Batch 444/2000
loss: 0.6607649922370911
---------------
Batch 445/2000
loss: 0.6221215724945068
---------------
Batch 446/2000
loss: 0.5737206935882568
---------------
Batch 447/2000
loss: 0.6323208808898926
---------------
Batch 448/2000
loss: 0.5565494298934937
---------------
Batch 449/2000
loss: 0.5676344037055969
---------------
Batch 450/2000
loss: 0.6489206552505493
---------------
Batch 451/2000
loss: 0.6566251516342163
---------------
Batch 452/2000
loss: 0.6228827834129333
---------------
Batch 453/2000
loss: 0.6128168702125549
---------------
Batch 454/2000
loss: 0.6682906150817871
---------------
Batch 455/2000
loss: 0.6075890064239502
---------------
Batch 456/2000
loss: 0.6841427087783813
---------------
Batch 457/2000
loss: 0.6534552574157715
---------------
Batch 458/2000
loss: 0.5676799416542053
---------------
Batch 459/2000
loss: 0.6042412519454956
---------------
Batch 460/2000
loss: 0.5738989114761353
---------------
Batch 461/2000
loss: 0.655882000

loss: 0.6108366847038269
---------------
Batch 591/2000
loss: 0.6106334924697876
---------------
Batch 592/2000
loss: 0.5977097153663635
---------------
Batch 593/2000
loss: 0.6267420053482056
---------------
Batch 594/2000
loss: 0.5975685119628906
---------------
Batch 595/2000
loss: 0.5523616075515747
---------------
Batch 596/2000
loss: 0.595294177532196
---------------
Batch 597/2000
loss: 0.6569606065750122
---------------
Batch 598/2000
loss: 0.6467339992523193
---------------
Batch 599/2000
loss: 0.5943281054496765
---------------
Batch 600/2000
loss: 0.5756601095199585
---------------
Batch 601/2000
loss: 0.603447675704956
---------------
Batch 602/2000
loss: 0.7109090089797974
---------------
Batch 603/2000
loss: 0.6349321603775024
---------------
Batch 604/2000
loss: 0.6427400708198547
---------------
Batch 605/2000
loss: 0.6305657029151917
---------------
Batch 606/2000
loss: 0.6241533160209656
---------------
Batch 607/2000
loss: 0.5856913328170776
---------------
Batch 608

...

---------------
Batch 1342/2000
loss: 0.4733424484729767
---------------
Batch 1343/2000
loss: 0.4038797616958618
---------------
Batch 1344/2000
loss: 0.5027403831481934
---------------
Batch 1345/2000
loss: 0.45896464586257935
---------------
Batch 1346/2000
loss: 0.38370710611343384
---------------
Batch 1347/2000
loss: 0.39601507782936096
---------------
Batch 1348/2000
loss: 0.3923603892326355
---------------
Batch 1349/2000
loss: 0.3848918676376343
---------------
Batch 1350/2000
loss: 0.4061335623264313
---------------
Batch 1351/2000
loss: 0.39575058221817017
---------------
Batch 1352/2000
loss: 0.3293887972831726
---------------
Batch 1353/2000
loss: 0.5182395577430725
---------------
Batch 1354/2000
loss: 0.4133782386779785
---------------
Batch 1355/2000
loss: 0.5622336268424988
---------------
Batch 1356/2000
loss: 0.49310168623924255
---------------
Batch 1357/2000
loss: 0.38261356949806213
---------------
Batch 1358/2000
loss: 0.44699665904045105
---------------
Batch 13

---------------
Batch 1485/2000
loss: 0.5620663166046143
---------------
Batch 1486/2000
loss: 0.5168586373329163
---------------
Batch 1487/2000
loss: 0.4407315254211426
---------------
Batch 1488/2000
loss: 0.5314880609512329
---------------
Batch 1489/2000
loss: 0.49339693784713745
---------------
Batch 1490/2000
loss: 0.4980525076389313
---------------
Batch 1491/2000
loss: 0.4650198817253113
---------------
Batch 1492/2000
loss: 0.5242907404899597
---------------
Batch 1493/2000
loss: 0.5141793489456177
---------------
Batch 1494/2000
loss: 0.5508301258087158
---------------
Batch 1495/2000
loss: 0.4944137930870056
---------------
Batch 1496/2000
loss: 0.44594812393188477
---------------
Batch 1497/2000
loss: 0.5378209948539734
---------------
Batch 1498/2000
loss: 0.4226461350917816
---------------
Batch 1499/2000
loss: 0.5563392639160156
---------------
Batch 1500/2000
loss: 0.5789744257926941
---------------
Batch 1501/2000
loss: 0.485836386680603
---------------
Batch 1502/200

---------------
Batch 1628/2000
loss: 0.38129326701164246
---------------
Batch 1629/2000
loss: 0.3409528434276581
---------------
Batch 1630/2000
loss: 0.40451717376708984
---------------
Batch 1631/2000
loss: 0.43180912733078003
---------------
Batch 1632/2000
loss: 0.34344807267189026
---------------
Batch 1633/2000
loss: 0.41392749547958374
---------------
Batch 1634/2000
loss: 0.56797194480896
---------------
Batch 1635/2000
loss: 0.3632048964500427
---------------
Batch 1636/2000
loss: 0.4870959222316742
---------------
Batch 1637/2000
loss: 0.3337767422199249
---------------
Batch 1638/2000
loss: 0.3575890064239502
---------------
Batch 1639/2000
loss: 0.3676609992980957
---------------
Batch 1640/2000
loss: 0.47738125920295715
---------------
Batch 1641/2000
loss: 0.41070109605789185
---------------
Batch 1642/2000
loss: 0.3939582109451294
---------------
Batch 1643/2000
loss: 0.4236797094345093
---------------
Batch 1644/2000
loss: 0.5034511685371399
---------------
Batch 1645

---------------
Batch 1771/2000
loss: 0.33201882243156433
---------------
Batch 1772/2000
loss: 0.33397096395492554
---------------
Batch 1773/2000
loss: 0.4015088379383087
---------------
Batch 1774/2000
loss: 0.4014308452606201
---------------
Batch 1775/2000
loss: 0.39891090989112854
---------------
Batch 1776/2000
loss: 0.38965868949890137
---------------
Batch 1777/2000
loss: 0.4983040988445282
---------------
Batch 1778/2000
loss: 0.431289941072464
---------------
Batch 1779/2000
loss: 0.4711341857910156
---------------
Batch 1780/2000
loss: 0.3839718699455261
---------------
Batch 1781/2000
loss: 0.4383738338947296
---------------
Batch 1782/2000
loss: 0.5653833150863647
---------------
Batch 1783/2000
loss: 0.3620198667049408
---------------
Batch 1784/2000
loss: 0.40493836998939514
---------------
Batch 1785/2000
loss: 0.40853604674339294
---------------
Batch 1786/2000
loss: 0.4279991388320923
---------------
Batch 1787/2000
loss: 0.3604671359062195
---------------
Batch 1788

Batch 1914/2000
loss: 0.38430333137512207
---------------
Batch 1915/2000
loss: 0.40584883093833923
---------------
Batch 1916/2000
loss: 0.34973108768463135
---------------
Batch 1917/2000
loss: 0.4237797260284424
---------------
Batch 1918/2000
loss: 0.3920929431915283
---------------
Batch 1919/2000
loss: 0.46310746669769287
---------------
Batch 1920/2000
loss: 0.3703821897506714
---------------
Batch 1921/2000
loss: 0.44753092527389526
---------------
Batch 1922/2000
loss: 0.49133622646331787
---------------
Batch 1923/2000
loss: 0.5059257745742798
---------------
Batch 1924/2000
loss: 0.37881579995155334
---------------
Batch 1925/2000
loss: 0.364899605512619
---------------
Batch 1926/2000
loss: 0.39457306265830994
---------------
Batch 1927/2000
loss: 0.3695361018180847
---------------
Batch 1928/2000
loss: 0.4284263551235199
---------------
Batch 1929/2000
loss: 0.3835221529006958
---------------
Batch 1930/2000
loss: 0.37938937544822693
---------------
Batch 1931/2000
loss: 0

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [2]:
import torch
import torch.nn as nn

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(